In [ ]:
import sys
!{sys.executable} -m pip install ultralytics
!{sys.executable} -m pip install supervision

from ultralytics import YOLO
import supervision as sv
import cv2
import pandas as pd
import numpy as np
import time
from supervision.metrics.mean_average_precision import MeanAveragePrecision
from tqdm import tqdm

# -------------------------------
# Setup
# -------------------------------
VIDEO_PATH = "/content/EV20250426-144029-000346F.mp4"   # 🔁 change this to your input video path
OUTPUT_VIDEO = "tracked_output.mp4"
CSV_OUTPUT = "annotations_yolov8_bytetrack.csv"
SKIP_FRAMES = 10   # ✅ process every 10th frame

# Load YOLOv8 + tracker
model = YOLO("yolov8n.pt")  # use nano for faster inference; replace with yolov8s.pt if needed
tracker = sv.ByteTrack()
box_annotator = sv.BoxAnnotator(thickness=2)
label_annotator = sv.LabelAnnotator(text_scale=0.5)

# Video setup
cap = cv2.VideoCapture(VIDEO_PATH)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(OUTPUT_VIDEO, cv2.VideoWriter_fourcc(*"mp4v"), fps / SKIP_FRAMES, (width, height))

frame_idx = 0
processed_count = 0
frame_times = []
annotation_data = []
map_metric = MeanAveragePrecision()

# -------------------------------
# Main Loop (with frame skipping)
# -------------------------------
for _ in tqdm(range(total_frames), desc="Annotating Video"):
    ret, frame = cap.read()
    if not ret:
        break

    # Skip frames that aren’t multiple of SKIP_FRAMES
    if frame_idx % SKIP_FRAMES != 0:
        frame_idx += 1
        continue

    t0 = time.time()
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    tracked = tracker.update_with_detections(detections)
    t1 = time.time()
    frame_times.append(t1 - t0)

    # Update pseudo mAP
    if len(detections) > 0 and len(tracked) > 0:
        map_metric.update(detections, tracked)

    # Save data for CSV
    for box, cls, track_id in zip(tracked.xyxy, tracked.class_id, tracked.tracker_id):
        x1, y1, x2, y2 = box
        annotation_data.append({
            "frame": frame_idx,
            "track_id": int(track_id),
            "class": model.model.names[int(cls)],
            "x1": float(x1), "y1": float(y1),
            "x2": float(x2), "y2": float(y2)
        })

    # Draw boxes + labels
    labels = [
        f"{model.model.names[c]} #{tid}"
        for c, tid in zip(tracked.class_id, tracked.tracker_id)
    ]
    frame = box_annotator.annotate(scene=frame, detections=tracked)
    frame = label_annotator.annotate(scene=frame, detections=tracked, labels=labels)

    out.write(frame)
    processed_count += 1
    frame_idx += 1

cap.release()
out.release()

# -------------------------------
# Save CSV + Metrics
# -------------------------------
df = pd.DataFrame(annotation_data)
df.to_csv(CSV_OUTPUT, index=False)

metrics = map_metric.compute()
# -------------------------------
# Save CSV + Metrics (fixed)
# -------------------------------
df = pd.DataFrame(annotation_data)
df.to_csv(CSV_OUTPUT, index=False)

metrics = map_metric.compute()

# Extract values safely (handles different versions)
map_50 = metrics.get("map_50", None) if isinstance(metrics, dict) else getattr(metrics, "map_50", None)
map_75 = metrics.get("map_75", None) if isinstance(metrics, dict) else getattr(metrics, "map_75", None)
map_5095 = metrics.get("map_50_95", None) if isinstance(metrics, dict) else getattr(metrics, "map_50_95", None)

avg_time = np.mean(frame_times)
fps_value = 1 / avg_time if avg_time > 0 else 0

print("\n📊 --- COMPARATIVE METRICS ---")
print(f"Total Frames in Video: {frame_idx}")
print(f"Frames Processed: {processed_count} (every {SKIP_FRAMES}th frame)")
print(f"Average Time per Frame: {avg_time:.4f} sec")
print(f"Annotation Speed: {fps_value:.2f} FPS (approx)")
if map_50 is not None:
    print(f"mAP@50: {map_50:.4f}")
if map_75 is not None:
    print(f"mAP@75: {map_75:.4f}")
if map_5095 is not None:
    print(f"mAP@50–95: {map_5095:.4f}")
print(f"📁 CSV saved as: {CSV_OUTPUT}")
print(f"🎥 Annotated video saved as: {OUTPUT_VIDEO}")
print("✅ YOLOv8 + ByteTrack Annotation Completed Successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 4.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Annotating Video: 100%|██████████| 945/945 [00:58<00:00, 16.28it/s]



📊 --- COMPARATIVE METRICS ---
Total Frames in Video: 945
Frames Processed: 95 (every 10th frame)
Average Time per Frame: 0.1742 sec
Annotation Speed: 5.74 FPS (approx)
📁 CSV saved as: annotations_yolov8_bytetrack.csv
🎥 Annotated video saved as: tracked_output.mp4
✅ YOLOv8 + ByteTrack Annotation Completed Successfully!


In [ ]:
# -------------------------------
# Simple IoU-based evaluation
# -------------------------------
def iou(boxA, boxB):
    xA, yA = max(boxA[0], boxB[0]), max(boxA[1], boxB[1])
    xB, yB = min(boxA[2], boxB[2]), min(boxA[3], boxB[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    union = (boxA[2]-boxA[0])*(boxA[3]-boxA[1]) + (boxB[2]-boxB[0])*(boxB[3]-boxB[1]) - inter
    return inter / union if union > 0 else 0

tp, fp, fn = 0, 0, 0
threshold = 0.5   # <-- correctly defined here

for f in sorted(df_gt["frame"].unique()):
    gt_frame = df_gt[df_gt["frame"] == f]
    pred_frame = df_pred[df_pred["frame"] == f]
    matched = set()
    for _, gt in gt_frame.iterrows():
        gt_box = [gt.x1, gt.y1, gt.x2, gt.y2]
        best_iou, match_j = 0, -1
        for j, pr in pred_frame.iterrows():
            pr_box = [pr.x1, pr.y1, pr.x2, pr.y2]
            score = iou(gt_box, pr_box)
            if score > best_iou:
                best_iou, match_j = score, j
        if best_iou >= threshold:
            tp += 1
            matched.add(match_j)
        else:
            fn += 1
    fp += len(pred_frame) - len(matched)

precision = tp / (tp + fp + 1e-9)
recall = tp / (tp + fn + 1e-9)
f1 = 2 * precision * recall / (precision + recall + 1e-9)
avg_time = np.mean(times) if len(times) > 0 else 0
fps_value = 1 / avg_time if avg_time > 0 else 0

# -------------------------------
# Print results
# -------------------------------
print("\n📊 --- PERFORMANCE METRICS ---")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"Avg Inference Time: {avg_time:.4f}s")
print(f"Speed: {fps_value:.2f} FPS")
print(f"📁 CSV saved as: {CSV_OUTPUT}")
print(f"🎥 Annotated video saved as: {OUTPUT_VIDEO}")
print("✅ YOLOv8 + ByteTrack annotation & evaluation completed.")



📊 --- PERFORMANCE METRICS ---
Precision: 0.0000
Recall:    0.0000
F1 Score:  0.0000
Avg Inference Time: 0.0000s
Speed: 0.00 FPS
📁 CSV saved as: detections_yolov8_bytetrack.csv
🎥 Annotated video saved as: tracked_output.mp4
✅ YOLOv8 + ByteTrack annotation & evaluation completed.


In [ ]:
from google.colab import files

print("📁 Upload your video file...")
uploaded = files.upload()

# Get the uploaded filename automatically
VIDEO_PATH = list(uploaded.keys())[0]
print(f"✔ Video uploaded: {VIDEO_PATH}")


📁 Upload your video file...


Saving EV20250426-144029-000346F.mp4 to EV20250426-144029-000346F (1).mp4
✔ Video uploaded: EV20250426-144029-000346F (1).mp4
